In [12]:
import sqlite3
import pandas as pd

In [13]:
con = sqlite3.connect(r'D:\prime_bot_dev\Prime-CIB\CIB_BOT_Delta-Screening\output\cib_delta_screening.db')
cur = con.cursor()

query = f'''
        select * from  cib_users
        where Status = 'new';
        '''
con.row_factory = sqlite3.Row
data = con.execute(query).fetchall()
if data:
    temp_value = [{str(key): item[key] for key in item.keys()} for item in data]
    

In [15]:
df = pd.DataFrame(temp_value)

In [16]:
df.head()

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,IndianEmbassyDetails,BlackLists,Account_Nature,Status
0,JAISARI RANA,2043-02-14,F,MODAR RANA,"['0663046-844|2063-11-09|Banke', '0663046/844|...",,,,602298847|2071-04-22|Banke,nan,,Bank and Financial Institutions|87349|2080-06-...,Individual,new
1,ASHOK PARAJULI,2039-04-29,M,PASHUPATI PARAJULI,80209|2055-11-26|Kaski,,,,613715562|2077-06-06|Kaski,nan,,Bank and Financial Institutions|87356|2080-06-...,Individual,new
2,SANJAYA CHAUDHARY,2049-05-27,M,RAMESH PRASAD CHAUDHARY,051051/19|2066-10-06|Morang,,,,,nan,,Bank and Financial Institutions|87353|2080-06-...,Individual,new
3,RAMESH PRASAD CHOUDHARI,2016-03-28,M,KHUSHI LAL THARU,05/1069|2039-03-28|Morang,,,,,nan,,Bank and Financial Institutions|87353|2080-06-...,Individual,new
4,CHAKRA BAHADUR BAM (BOGATI),2037-10-04,M,BHIM BAHADUR BOGATI,32444|2056-11-13|Kailali,,,,,nan,,Bank and Financial Institutions|87347|2080-06-...,Individual,new


In [17]:
if 'Individual' in df['Account_Nature'].values:
    individual_df = df[df['Account_Nature'] == 'Individual']
    # Do something with individual_df
elif 'Institutions' in df['Account_Nature'].values:
    institutions_df = df[df['Account_Nature'] == 'Institutions']
    # Do something with institutions_df
else:
    print('None type of Nature')


In [18]:
individual_df.head()

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,IndianEmbassyDetails,BlackLists,Account_Nature,Status
0,JAISARI RANA,2043-02-14,F,MODAR RANA,"['0663046-844|2063-11-09|Banke', '0663046/844|...",,,,602298847|2071-04-22|Banke,nan,,Bank and Financial Institutions|87349|2080-06-...,Individual,new
1,ASHOK PARAJULI,2039-04-29,M,PASHUPATI PARAJULI,80209|2055-11-26|Kaski,,,,613715562|2077-06-06|Kaski,nan,,Bank and Financial Institutions|87356|2080-06-...,Individual,new
2,SANJAYA CHAUDHARY,2049-05-27,M,RAMESH PRASAD CHAUDHARY,051051/19|2066-10-06|Morang,,,,,nan,,Bank and Financial Institutions|87353|2080-06-...,Individual,new
3,RAMESH PRASAD CHOUDHARI,2016-03-28,M,KHUSHI LAL THARU,05/1069|2039-03-28|Morang,,,,,nan,,Bank and Financial Institutions|87353|2080-06-...,Individual,new
4,CHAKRA BAHADUR BAM (BOGATI),2037-10-04,M,BHIM BAHADUR BOGATI,32444|2056-11-13|Kailali,,,,,nan,,Bank and Financial Institutions|87347|2080-06-...,Individual,new


In [10]:
client_table_df = pd.read_excel(r'D:\prime_bot_dev\Prime-CIB\CIB_BOT_Delta-Screening\output\client_tablee.xlsx')

In [30]:
import Levenshtein as lev
import pandas as pd
import ast

# Sample DataFrames (replace these with your actual DataFrames)
for index, item in individual_df['CitizenshipDetails'].items():
    item = str(item)
    
    # Check if item contains commas
    if ',' in item:
        citizen_list = ast.literal_eval(item)
        if isinstance(citizen_list, list):
            for data in citizen_list:
                # Check if '|' exists before splitting
                if '|' in data:
                    citizen_number = data.split('|')[1]
                    individual_df.at[index, 'CitizenshipNumber'] = citizen_number
                else:
                    print(f"Invalid format in item: {data}")
    else:
        # Check if '|' exists before splitting
        if '|' in item:
            citizen_number = item.split('|')[1]
            # print(citizen_number)
            individual_df.at[index, 'CitizenshipNumber'] = citizen_number
        else:
            print(f"Invalid format in item: {item}")
individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
individual_df['Name'] = individual_df['Name'].astype(str)
individual_df['FatherName'] = individual_df['FatherName'].astype(str)
individual_df['DOB'] = individual_df['DOB'].astype(str)

client_table_df['name'] = client_table_df['name'].astype(str)
client_table_df['fathersname'] = client_table_df['fathersname'].astype(str)
client_table_df['citizenshipno'] = client_table_df['citizenshipno'].astype(str)
client_table_df['dateofbirth'] = client_table_df['dateofbirth'].astype(str)

# Function to calculate Levenshtein ratio and convert to percentage
def calculate_similarity(row, client_data):
    try:
        similarity_scores = [lev.ratio(str(row), str(client_item)) for client_item in client_data]
        avg_similarity = (sum(similarity_scores) / len(similarity_scores)) * 100
        return avg_similarity
    except Exception as e:
        print(f"Error processing item: {row}. Error: {e}")
        return 0  # or any other default value indicating an error

# Calculate similarities using apply function
cib_similarity = pd.DataFrame({
    'name_percent': individual_df['Name'].apply(lambda x: calculate_similarity(x, client_table_df['name'])),
    'father_name_percent': individual_df['FatherName'].apply(lambda x: calculate_similarity(x, client_table_df['fathersname'])),
    'citizenship_percent': individual_df['CitizenshipNumber'].apply(lambda x: calculate_similarity(x, client_table_df['citizenshipno'])),
    'dob_percent': individual_df['DOB'].apply(lambda x: calculate_similarity(x, client_table_df['dateofbirth']))
})

# cbs_similarity = pd.DataFrame({
#     'NameSimilarity': client_table_df['name'].apply(lambda x: calculate_similarity(x, individual_df['Name'])),
#     'FatherNameSimilarity': client_table_df['fathersname'].apply(lambda x: calculate_similarity(x, individual_df['FatherName'])),
#     'CitizenshipSimilarity': client_table_df['citizenshipno'].apply(lambda x: calculate_similarity(x, individual_df['CitizenshipNumber'])),
#     'DOBSimilarity': client_table_df['dateofbirth'].apply(lambda x: calculate_similarity(x, individual_df['DOB']))
# })

# Rest of your code...


Invalid format in item: 


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\1410287645.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\1410287645.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['Name'] = individual_df['Name'].astype(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\1410287645.py:30: SettingWithCopyWarning: 
A value is trying to be s

In [32]:
cib_similarity.shape

(212, 4)

In [33]:
cib_similarity.head()

,name_percent,father_name_percent,citizenship_percent,dob_percent
0,37.923036,31.219171,19.670453,26.482759
1,34.831691,30.822782,18.434855,27.681379
2,37.973428,35.677158,16.703090,27.601379
3,38.856572,31.938362,18.472870,29.150345
4,34.057169,33.008437,18.369485,31.485517


In [26]:
similarity_df_apply_cbs.shape

(5000, 4)

In [38]:
import Levenshtein as lev
import pandas as pd
import ast

# Sample DataFrames (replace these with your actual DataFrames)
for index, item in individual_df['CitizenshipDetails'].items():
    item = str(item)
    
    # Check if item contains commas
    if ',' in item:
        citizen_list = ast.literal_eval(item)
        if isinstance(citizen_list, list):
            for data in citizen_list:
                # Check if '|' exists before splitting
                if '|' in data:
                    citizen_number = data.split('|')[1]
                    individual_df.at[index, 'CitizenshipNumber'] = citizen_number
                else:
                    print(f"Invalid format in item: {data}")
    else:
        # Check if '|' exists before splitting
        if '|' in item:
            citizen_number = item.split('|')[1]
            # print(citizen_number)
            individual_df.at[index, 'CitizenshipNumber'] = citizen_number
        else:
            print(f"Invalid format in item: {item}")
individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
individual_df['Name'] = individual_df['Name'].astype(str)
individual_df['FatherName'] = individual_df['FatherName'].astype(str)
individual_df['DOB'] = individual_df['DOB'].astype(str)

client_table_df['name'] = client_table_df['name'].astype(str)
client_table_df['fathersname'] = client_table_df['fathersname'].astype(str)
client_table_df['citizenshipno'] = client_table_df['citizenshipno'].astype(str)
client_table_df['dateofbirth'] = client_table_df['dateofbirth'].astype(str)

# Function to calculate Levenshtein ratio and convert to percentage
def calculate_similarity(row, client_data):
    try:
        similarity_scores = [lev.ratio(str(row), str(client_item)) for client_item in client_data]
        avg_similarity = (sum(similarity_scores) / len(similarity_scores)) * 100
        return avg_similarity
    except Exception as e:
        print(f"Error processing item: {row}. Error: {e}")
        return 0  # or any other default value indicating an error

# Calculate similarities using apply function
individual_df['name_percent'] = individual_df['Name'].apply(lambda x: calculate_similarity(x, client_table_df['name']))
individual_df['father_name_percent'] = individual_df['FatherName'].apply(lambda x: calculate_similarity(x, client_table_df['fathersname']))
individual_df['citizenship_percent'] = individual_df['CitizenshipNumber'].apply(lambda x: calculate_similarity(x, client_table_df['citizenshipno']))
individual_df['dob_percent'] = individual_df['DOB'].apply(lambda x: calculate_similarity(x, client_table_df['dateofbirth']))


# Calculate weighted similarity percentages
individual_df['weightage_name'] = individual_df['name_percent'] * Weitage_data['Name'] / 100
individual_df['weightage_fathername'] = individual_df['father_name_percent'] * Weitage_data['father_name'] / 100
individual_df['weightage_dob'] = individual_df['dob_percent'] * Weitage_data['dob'] / 100
individual_df['weightage_citizenship_no'] = individual_df['citizenship_percent'] * Weitage_data['citizenship_no'] / 100


Invalid format in item: 


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\393429975.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\393429975.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['Name'] = individual_df['Name'].astype(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\393429975.py:30: SettingWithCopyWarning: 
A value is trying to be set 

In [39]:
individual_df.head()

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,IndianEmbassyDetails,BlackLists,Account_Nature,Status,CitizenshipNumber,index,name_percent,father_name_percent,citizenship_percent,dob_percent
0,JAISARI RANA,2043-02-14,F,MODAR RANA,"['0663046-844|2063-11-09|Banke', '0663046/844|...",,,,602298847|2071-04-22|Banke,nan,,Bank and Financial Institutions|87349|2080-06-...,Individual,new,2063-11-09,0,37.923036,31.219171,19.670453,26.482759
1,ASHOK PARAJULI,2039-04-29,M,PASHUPATI PARAJULI,80209|2055-11-26|Kaski,,,,613715562|2077-06-06|Kaski,nan,,Bank and Financial Institutions|87356|2080-06-...,Individual,new,2055-11-26,1,34.831691,30.822782,18.434855,27.681379
2,SANJAYA CHAUDHARY,2049-05-27,M,RAMESH PRASAD CHAUDHARY,051051/19|2066-10-06|Morang,,,,,nan,,Bank and Financial Institutions|87353|2080-06-...,Individual,new,2066-10-06,2,37.973428,35.677158,16.703090,27.601379
3,RAMESH PRASAD CHOUDHARI,2016-03-28,M,KHUSHI LAL THARU,05/1069|2039-03-28|Morang,,,,,nan,,Bank and Financial Institutions|87353|2080-06-...,Individual,new,2039-03-28,3,38.856572,31.938362,18.472870,29.150345
4,CHAKRA BAHADUR BAM (BOGATI),2037-10-04,M,BHIM BAHADUR BOGATI,32444|2056-11-13|Kailali,,,,,nan,,Bank and Financial Institutions|87347|2080-06-...,Individual,new,2056-11-13,4,34.057169,33.008437,18.369485,31.485517


In [ ]:
individual_df['name_percent'] = individual_df['Name'].apply(lambda x: calculate_similarity(x, client_table_df['name']))
individual_df['father_name_percent'] = individual_df['FatherName'].apply(lambda x: calculate_similarity(x, client_table_df['fathersname']))
individual_df['citizenship_percent'] = individual_df['CitizenshipNumber'].apply(lambda x: calculate_similarity(x, client_table_df['citizenshipno']))
individual_df['dob_percent'] = individual_df['DOB'].apply(lambda x: calculate_similarity(x, client_table_df['dateofbirth']))

In [40]:
Weitage_data = {
    'Name':40,
    'citizenship_no':20,
    'father_name':20,
    'dob':20
}


In [62]:
import Levenshtein as lev
import pandas as pd
import ast


Weitage_data = {
    'Name':40,
    'citizenship_no':20,
    'father_name':20,
    'dob':20
}
# Sample DataFrames (replace these with your actual DataFrames)
for index, item in individual_df['CitizenshipDetails'].items():
    item = str(item)
    
    # Check if item contains commas
    if ',' in item:
        citizen_list = ast.literal_eval(item)
        if isinstance(citizen_list, list):
            for data in citizen_list:
                # Check if '|' exists before splitting
                if '|' in data:
                    citizen_number = data.split('|')[1]
                    individual_df.at[index, 'CitizenshipNumber'] = citizen_number
                else:
                    print(f"Invalid format in item: {data}")
    else:
        # Check if '|' exists before splitting
        if '|' in item:
            citizen_number = item.split('|')[1]
            # print(citizen_number)
            individual_df.at[index, 'CitizenshipNumber'] = citizen_number
        else:
            print(f"Invalid format in item: {item}")
individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
individual_df['Name'] = individual_df['Name'].astype(str)
individual_df['FatherName'] = individual_df['FatherName'].astype(str)
individual_df['DOB'] = individual_df['DOB'].astype(str)

client_table_df['name'] = client_table_df['name'].astype(str)
client_table_df['fathersname'] = client_table_df['fathersname'].astype(str)
client_table_df['citizenshipno'] = client_table_df['citizenshipno'].astype(str)
client_table_df['dateofbirth'] = client_table_df['dateofbirth'].astype(str)

# Function to calculate Levenshtein ratio and convert to percentage
def calculate_similarity(row, client_data):
    row = row.lower()
    try:
        similarity_scores = [lev.ratio(str(row), str(client_item.lower())) for client_item in client_data]
        avg_similarity = (sum(similarity_scores) / len(similarity_scores)) * 100
        return avg_similarity
    except Exception as e:
        print(f"Error processing item: {row}. Error: {e}")
        return 0  # or any other default value indicating an error

# Calculate similarities using apply function
individual_df['name_percent'] = individual_df['Name'].apply(lambda x: calculate_similarity(x, client_table_df['name']))
individual_df['father_name_percent'] = individual_df['FatherName'].apply(lambda x: calculate_similarity(x, client_table_df['fathersname']))
individual_df['citizenship_percent'] = individual_df['CitizenshipNumber'].apply(lambda x: calculate_similarity(x, client_table_df['citizenshipno']))
individual_df['dob_percent'] = individual_df['DOB'].apply(lambda x: calculate_similarity(x, client_table_df['dateofbirth']))


# Calculate weighted similarity percentages
individual_df['weightage_name'] = individual_df['name_percent'] * Weitage_data['Name'] / 100
individual_df['weightage_fathername'] = individual_df['father_name_percent'] * Weitage_data['father_name'] / 100
individual_df['weightage_dob'] = individual_df['dob_percent'] * Weitage_data['dob'] / 100
individual_df['weightage_citizenship_no'] = individual_df['citizenship_percent'] * Weitage_data['citizenship_no'] / 100

individual_df['total_weight'] = individual_df['weightage_name'] + individual_df['weightage_fathername'] + individual_df['weightage_dob'] + individual_df['weightage_citizenship_no']



Invalid format in item: 


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\1186336302.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\1186336302.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['Name'] = individual_df['Name'].astype(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13272\1186336302.py:37: SettingWithCopyWarning: 
A value is trying to be s

In [44]:
individual_df.head()

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,...,index,name_percent,father_name_percent,citizenship_percent,dob_percent,weightage_name,weightage_fathername,weightage_dob,weightage_citizenship_no,total_weight
0,JAISARI RANA,2043-02-14,F,MODAR RANA,"['0663046-844|2063-11-09|Banke', '0663046/844|...",,,,602298847|2071-04-22|Banke,nan,...,0,37.923036,31.219171,19.670453,26.482759,15.169214,6.243834,5.296552,3.934091,30.643691
1,ASHOK PARAJULI,2039-04-29,M,PASHUPATI PARAJULI,80209|2055-11-26|Kaski,,,,613715562|2077-06-06|Kaski,nan,...,1,34.831691,30.822782,18.434855,27.681379,13.932677,6.164556,5.536276,3.686971,29.320480
2,SANJAYA CHAUDHARY,2049-05-27,M,RAMESH PRASAD CHAUDHARY,051051/19|2066-10-06|Morang,,,,,nan,...,2,37.973428,35.677158,16.703090,27.601379,15.189371,7.135432,5.520276,3.340618,31.185697
3,RAMESH PRASAD CHOUDHARI,2016-03-28,M,KHUSHI LAL THARU,05/1069|2039-03-28|Morang,,,,,nan,...,3,38.856572,31.938362,18.472870,29.150345,15.542629,6.387672,5.830069,3.694574,31.454944
4,CHAKRA BAHADUR BAM (BOGATI),2037-10-04,M,BHIM BAHADUR BOGATI,32444|2056-11-13|Kailali,,,,,nan,...,4,34.057169,33.008437,18.369485,31.485517,13.622868,6.601687,6.297103,3.673897,30.195556


In [63]:
filtered_individual_df = individual_df[individual_df['total_weight'] > 40]


In [64]:
filtered_individual_df

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,...,index,name_percent,father_name_percent,citizenship_percent,dob_percent,weightage_name,weightage_fathername,weightage_dob,weightage_citizenship_no,total_weight


In [65]:
df = pd.read_excel(r'D:\prime_bot_dev\Prime-CIB\CIB_BOT_Delta-Screening\weitage.xlsx')

In [66]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'typeofclient', 'clientcategory',
       'ownershiptype', 'clientcode', 'name', 'firstname', 'middlename',
       'lastname', 'jointname', 'address1', 'dateofbirth', 'citizenshipno',
       'citizendistrict', 'fathersname', 'gfathersname', 'spousename',
       'clientstatus', 'mobileno', 'phone', 'key_risk_grade', 'reason',
       'pannumber', 'regnum', 'name_percent', 'name_weight',
       'fathersname_percent', 'fathersname_weight', 'dateofbirth_percent',
       'dateofbirth_weight', 'citizenshipno_percent', 'citizenshipno_weight'],
      dtype='object')

In [67]:
df.head()

,Unnamed: 0.1,Unnamed: 0,typeofclient,clientcategory,ownershiptype,clientcode,name,firstname,middlename,lastname,...,pannumber,regnum,name_percent,name_weight,fathersname_percent,fathersname_weight,dateofbirth_percent,dateofbirth_weight,citizenshipno_percent,citizenshipno_weight
0,0,0,1.0,NaN,001 Individual,1,DEEPAK ADHIKARI,DEEPAK,NaN,ADHIKARI,...,103513700.0,NaN,23.08,9.232,38.89,7.778,34.48,6.896,16.67,3.334
1,1,1,1.0,NaN,001 Individual,2,KAMAL KUMAR CHAUDHARY,KAMAL,KUMAR,CHAUDHARY,...,NaN,NaN,31.25,12.500,35.29,7.058,27.59,5.518,24.00,4.800
2,2,2,2.0,Oth,NaN,3,JIRI CAMPUS,JIRI CAMPUS,NaN,NaN,...,999999999.0,NaN,27.27,10.908,19.05,3.810,27.59,5.518,0.00,0.000
3,3,3,1.0,NaN,001 Individual,4,PRITY DHANA RAI,PRITY,DHANA,RAI,...,NaN,NaN,30.77,12.308,70.59,14.118,34.48,6.896,14.29,2.858
4,4,4,1.0,NaN,001 Individual,5,RONESH STHAPIT,RONESH,NaN,STHAPIT,...,NaN,NaN,16.00,6.400,30.00,6.000,27.59,5.518,0.00,0.000


In [19]:
import Levenshtein as lev
import jellyfish
import pandas as pd
import ast

Weitage_data = {
    'Name': 40,
    'citizenship_no': 20,
    'father_name': 20,
    'dob': 20
}

# Sample DataFrames (replace these with your actual DataFrames)
for index, item in individual_df['CitizenshipDetails'].items():
    item = str(item)
    
    # Check if item contains commas
    if ',' in item:
        citizen_list = ast.literal_eval(item)
        if isinstance(citizen_list, list):
            for data in citizen_list:
                # Check if '|' exists before splitting
                if '|' in data:
                    citizen_number = data.split('|')[1]
                    individual_df.at[index, 'CitizenshipNumber'] = citizen_number
                else:
                    print(f"Invalid format in item: {data}")
    else:
        # Check if '|' exists before splitting
        if '|' in item:
            citizen_number = item.split('|')[1]
            # print(citizen_number)
            individual_df.at[index, 'CitizenshipNumber'] = citizen_number
        else:
            print(f"Invalid format in item: {item}")
individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
individual_df['Name'] = individual_df['Name'].astype(str)
individual_df['FatherName'] = individual_df['FatherName'].astype(str)
individual_df['DOB'] = individual_df['DOB'].astype(str)

client_table_df['name'] = client_table_df['name'].astype(str)
client_table_df['fathersname'] = client_table_df['fathersname'].astype(str)
client_table_df['citizenshipno'] = client_table_df['citizenshipno'].astype(str)
client_table_df['dateofbirth'] = client_table_df['dateofbirth'].astype(str)
# Function to calculate similarity using Soundex and Levenshtein
def calculate_similarity(row, client_data, soundex=True):
    row = row.lower()
    try:
        if soundex:
            row_soundex = jellyfish.soundex(row)
            similarity_scores = [lev.ratio(row_soundex, jellyfish.soundex(str(client_item).lower())) for client_item in client_data]
        else:
            similarity_scores = [lev.ratio(str(row), str(client_item).lower()) for client_item in client_data]
        
        avg_similarity = (sum(similarity_scores) / len(similarity_scores)) * 100
        return avg_similarity
    except Exception as e:
        print(f"Error processing item: {row}. Error: {e}")
        return 0  # or any other default value indicating an error

# Calculate similarities using apply function with Soundex
individual_df['name_percent'] = individual_df['Name'].apply(lambda x: calculate_similarity(x, client_table_df['name']))
individual_df['father_name_percent'] = individual_df['FatherName'].apply(lambda x: calculate_similarity(x, client_table_df['fathersname']))
individual_df['citizenship_percent'] = individual_df['CitizenshipNumber'].apply(lambda x: calculate_similarity(x, client_table_df['citizenshipno']))
individual_df['dob_percent'] = individual_df['DOB'].apply(lambda x: calculate_similarity(x, client_table_df['dateofbirth']))

# Calculate weighted similarity percentages
individual_df['weightage_name'] = individual_df['name_percent'] * Weitage_data['Name'] / 100
individual_df['weightage_fathername'] = individual_df['father_name_percent'] * Weitage_data['father_name'] / 100
individual_df['weightage_dob'] = individual_df['dob_percent'] * Weitage_data['dob'] / 100
individual_df['weightage_citizenship_no'] = individual_df['citizenship_percent'] * Weitage_data['citizenship_no'] / 100

individual_df['total_weight'] = individual_df['weightage_name'] + individual_df['weightage_fathername'] + individual_df['weightage_dob'] + individual_df['weightage_citizenship_no']


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21980\3365693320.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df.at[index, 'CitizenshipNumber'] = citizen_number
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21980\3365693320.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['CitizenshipNumber'] = individual_df['CitizenshipNumber'].astype(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21980\3365693320.py:37: SettingWithCopyWarning: 
A value is trying to 

In [27]:
filtered_individual_df = individual_df[individual_df['total_weight'] > 50]
filtered_individual_df

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,...,CitizenshipNumber,name_percent,father_name_percent,citizenship_percent,dob_percent,weightage_name,weightage_fathername,weightage_dob,weightage_citizenship_no,total_weight
2,SANJAYA CHAUDHARY,2049-05-27,M,RAMESH PRASAD CHAUDHARY,051051/19|2066-10-06|Morang,,,,,nan,...,2066-10-06,37.815,33.910,70.855,77.060,15.126,6.782,15.412,14.171,51.491
5,BHIM KUMARI TAMANG,2030-05-03,F,NAR BAHADUR TAMANG,2026/8364|2054-11-05|Sindhuli,,,,,nan,...,2054-11-05,36.840,29.755,70.855,77.060,14.736,5.951,15.412,14.171,50.270
14,SHANKAR SHAH,2050-04-04,M,MOHAN SAHU SUDI,151090/16667|2068-08-17|Siraha,,,,,nan,...,2068-08-17,39.665,27.765,70.855,77.060,15.866,5.553,15.412,14.171,51.002
17,KRISHNA MAHATO,2054-03-18,M,BHIKHARI MAHTO KEWAT,34-01-73-00291|2073-01-09|Parsa,,,,,nan,...,2073-01-09,36.290,34.980,70.855,77.060,14.516,6.996,15.412,14.171,51.095
25,SACHIN K.C.,2055-09-07,M,BED BAHADUR KHATRI,43-01-71-06323|2071-01-25|Tanahun,,,,,nan,...,2071-01-25,40.755,21.340,70.855,77.060,16.302,4.268,15.412,14.171,50.153
26,PREM SINGH JAGARI,2045-01-07,M,JHUN SINGH JAGARI,723020/161|2063-11-14|Darchula,,,,,nan,...,2063-11-14,37.490,30.555,70.855,77.060,14.996,6.111,15.412,14.171,50.690
33,ANSHA BAHADUR YADAV,2036-01-20,M,NANDA AHIR,"['5177|2042-11-13|Rupandehi', '5177|2052-11-13...",,,,,nan,...,2052-11-13,36.305,36.925,70.855,77.060,14.522,7.385,15.412,14.171,51.490
35,SARASWATI KUMARI,2027-02-02,F,KISAN NARAYAN SAH,81164117|2045-02-02|Saptari,,,,,nan,...,2045-02-02,37.165,29.235,70.855,77.060,14.866,5.847,15.412,14.171,50.296
38,SOM PRASAD KAFLE,2021-11-12,M,INDRA PRASAD KAFLE,2547|2044-11-12|Dolakha,,,,,nan,...,2044-11-12,35.175,31.840,70.855,77.060,14.070,6.368,15.412,14.171,50.021
41,BINOD KUMAR MAHATO,2042-01-05,M,BHIKHARI MAHATO SUDI,193090/651|2063-10-16|Sarlahi,,,,,nan,...,2063-10-16,38.420,34.980,70.855,77.060,15.368,6.996,15.412,14.171,51.947


In [21]:
individual_df.head()

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,...,CitizenshipNumber,name_percent,father_name_percent,citizenship_percent,dob_percent,weightage_name,weightage_fathername,weightage_dob,weightage_citizenship_no,total_weight
0,JAISARI RANA,2043-02-14,F,MODAR RANA,"['0663046-844|2063-11-09|Banke', '0663046/844|...",,,,602298847|2071-04-22|Banke,nan,...,2063-11-09,24.380,20.355,70.855,77.06,9.752,4.071,15.412,14.171,43.406
1,ASHOK PARAJULI,2039-04-29,M,PASHUPATI PARAJULI,80209|2055-11-26|Kaski,,,,613715562|2077-06-06|Kaski,nan,...,2055-11-26,28.910,27.025,70.855,77.06,11.564,5.405,15.412,14.171,46.552
2,SANJAYA CHAUDHARY,2049-05-27,M,RAMESH PRASAD CHAUDHARY,051051/19|2066-10-06|Morang,,,,,nan,...,2066-10-06,37.815,33.910,70.855,77.06,15.126,6.782,15.412,14.171,51.491
3,RAMESH PRASAD CHOUDHARI,2016-03-28,M,KHUSHI LAL THARU,05/1069|2039-03-28|Morang,,,,,nan,...,2039-03-28,36.875,15.380,70.855,77.06,14.750,3.076,15.412,14.171,47.409
4,CHAKRA BAHADUR BAM (BOGATI),2037-10-04,M,BHIM BAHADUR BOGATI,32444|2056-11-13|Kailali,,,,,nan,...,2056-11-13,29.785,34.720,70.855,77.06,11.914,6.944,15.412,14.171,48.441


In [20]:
df['total_weight'] = df['weightage_name'] + df['weightage_fathername'] + df['dateofbirth_weight'] + df['citizenshipno_weight']


KeyError: 'name_weight'

In [72]:
filtered_df = df[df['total_weight'] > 60]
filtered_df.values


array([[4273, 4273, 1.0, nan, '001 Individual', 4356, 'BABITA GURUNG',
        'BABITA ', nan, 'GURUNG', nan, 'GILUNG 3',
        datetime.datetime(1972, 9, 17, 0, 0), '11250', 'LAM',
        'BAM BAHADUR GURUNG', 'GANESH BAHADUR GURUNG',
        'KUMAR PRASAD ADHIKARI', 'A', '9803197100', '4385510', 'LR', nan,
        nan, nan, 75.0, 30.0, 88.89, 17.778, 34.48, 6.896, 40.0, 8.0,
        62.674]], dtype=object)

In [3]:
import ast

# string2 = "['38705-2032']"
string2 = "Aayaan"
# Check if the string contains '[' character
if '[' in string2:
    string2_list = ast.literal_eval(string2)
    for item in string2_list:
        print(item)
else:
    string2 = string2
    print(string2)

Aayaan


In [7]:
import ast
try:
    string2_list = ast.literal_eval(string2)
    for item in string2_list:
        string2 = item
except (ValueError, SyntaxError):
    string2 = string2


In [8]:
import pandas as pd
import jellyfish

# Sample dataframes
df1 = pd.DataFrame({
    'Name': ['ram', 'sam', 'sita', 'gita', 'ram', 'pradip'],
    'DOB': ['1990-01-01', '1985-05-20', '1988-12-15', '1992-03-10', '1990-01-01', '1982-07-18']
})

df2 = pd.DataFrame({
    'Name': ['ram', 'sam', 'sita', 'gita', 'pradeep'],
    'DOB': ['1990-01-01', '1985-05-20', '1988-12-15', '1992-03-10', '1982-07-18']
})

# Function to calculate Levenshtein similarity using Soundex condition
def calculate_similarity(row1, row2):
    if jellyfish.soundex(row1['Name']) == jellyfish.soundex(row2['Name']):
        return jellyfish.levenshtein_distance(row1['Name'], row2['Name'])
    else:
        return None

# Calculate similarity and create a new dataframe with results
similarities = []
for index1, row1 in df1.iterrows():
    for index2, row2 in df2.iterrows():
        similarity = calculate_similarity(row1, row2)
        if similarity is not None:
            similarities.append({
                'Name1': row1['Name'],
                'Name2': row2['Name'],
                'Similarity': similarity
            })

result_df = pd.DataFrame(similarities)

# Display the result dataframe
print(result_df)


    Name1    Name2  Similarity
0     ram      ram           0
1     sam      sam           0
2    sita     sita           0
3    gita     gita           0
4     ram      ram           0
5  pradip  pradeep           2
